In [1]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = 'drive/MyDrive/imitate-retrieve-paraphrase'

Mounted at /content/drive


In [2]:
%pip install datasets
%pip install transformers
%pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.7 MB/s eta 0:00:00


In [3]:
from datasets import Dataset
import numpy as np
import nltk
import tqdm
import torch
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification

In [4]:
ds = Dataset.from_file(drive_path + '/medline_data.arrow')

In [5]:
train_data = {'text': [], 'label': []}
num_labels = 0
nltk.download('punkt')

for row in ds:
    text = row['output_aug']
    sentences = nltk.sent_tokenize(text, language = 'english')

    for idx, sentence in enumerate(sentences):
        num_labels = max(num_labels, idx + 1)
        train_data['text'].append(sentence)
        train_data['label'].append(idx)

train_data = Dataset.from_dict(train_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
print(num_labels)

8


In [6]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation = True)

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_train_data = train_data.map(preprocess_function, batched = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2076 [00:00<?, ? examples/s]

In [9]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = num_labels).to('cuda')

training_args = TrainingArguments(
    output_dir = drive_path + '/retriever',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    weight_decay = 0.01,
    save_strategy = 'no'
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_data,
    tokenizer = tokenizer,
    data_collator = data_collator
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
trainer.train()
trainer.save_model(drive_path + '/retriever/model')

Step,Training Loss
500,1.019800
